# Dumping radar data in a CF compliant format

This notebook describes how to create a netCDF file with the output of out radar data following the conventions described in http://cfconventions.org/documents.html V1.7.


For future reference, http://wiki.esipfed.org/index.php/NetCDF-CF_File_Examples_for_Satellite_Swath_Data
is an example on how to deal with satellite data.

In [ ]:
from netCDF4 import Dataset
from tdm.radar import utils
from datetime import datetime, timedelta
import numpy as np
import imageio
import os

In [ ]:
def get_image_data(path):
    im = imageio.imread(path)
    return im[:,:,0], im[:,:,3] == 255 # NOTE the mask is on channel 3!!!

def setattr(o, attrs):
    for k, v in attrs.items():
        o.setncattr(k, v)

def showattrs(o):
    for k in o.ncattrs():
        print(k, o.getncattr(k))

In [ ]:
root = './data/radar/cag01est2400/'
template = './data/radarsample/radarfootprint.tif'
sample_image_dt = '2018-05-01_23:01:05'
after = datetime(2018, 5, 1, 23, 0, 0)
before = datetime(2018, 5, 2, 16, 0, 0)
delta = timedelta(hours=1)

In [ ]:
oX, oY, pxlW, pxlH, cols, rows = utils.get_grid(template, unit='km')

In [ ]:
xpos = oX + pxlW * np.arange(0, cols)
ypos = oY + pxlH * np.arange(0, rows)

In [ ]:
radar_images = list(utils.get_raw_radar_images(root, after, before))
first_tstamp = radar_images[0][0]

In [ ]:
os.unlink('radar.nc')
r = Dataset('radar.nc', 'w')

In [ ]:
global_attributes = {
    'Conventions': 'CF-1.7',
    'title': 'Rainfall Radar acquisitions',
    'institution': 'AEN inc',
    'source': 'Radar XXX',
    'references': 'http://www.tdm-project.it/en/XXX/radar',
    'history': 'Conversion from raw data using XXXX'
}
setattr(r, global_attributes)

In [ ]:
showattrs(r)

In [ ]:
# See 4.4 time coordinate
timed = r.createDimension('time', None)
xd = r.createDimension('x', cols)
yd = r.createDimension('y', rows)

# See section 5.1 and https://code.mpimet.mpg.de/boards/1/topics/5765

times = r.createVariable('time', 'f4', (timed.name,))
setattr(times, {'long_name': 'time', 
               'units': 'minutes since %s UTC' % first_tstamp})

x = r.createVariable('x', 'f4', (xd.name,))
setattr(x, {'long_name': 'x coordinate of projection',
            'standard_name': 'projection_x_coordinate',
            'units': 'km'})

y = r.createVariable('y', 'f4', (yd.name,))
setattr(y, {'long_name': 'y coordinate of projection',
            'standard_name': 'projection_y_coordinate',
            'units': 'km'})

lat = r.createVariable('lat', 'f4', (xd.name, yd.name))
setattr(lat, {'long_name': 'latitude coordinate',
              'standard_name': 'latitude',
              'units': 'degrees_north'})

lon = r.createVariable('lon', 'f4', (xd.name, yd.name))
setattr(lat, {'long_name': 'longitude coordinate',
              'standard_name': 'longitude',
              'units': 'degrees_east'})

In [ ]:
# See section 5.6.1
crs = r.createVariable('crs', 'i4') # a dummy scalar used as anchor for the crs
setattr(crs, {
 'grid_mapping_name': 'transverse_mercator',
 'longitude_of_central_meridian': 9.0,
 'latitude_of_projection_origin': 0.0,
 'false_easting': 1500000.0,
 'false_northing': 0.0,
 'scale_factor_at_central_meridian': 0.9996,
 'semi_major_axis': 6378388.0,
 'inverse_flattening': 297,
 'projected_coordinate_system_name': 'EPSG:3003 Monte Mario / Italy zone 1',
 'geographic_coordinate_system_name': 'Monte Mario',
 'horizontal_datum_name':  'Monte_Mario',
 'reference_ellipsoid_name': 'International 1924',
 'prime_meridian_name': "Greenwich",
 'towgs84': [-104.1,-49.1,-9.9,0.971,-2.917,0.714,-11.68],
 'crs_wkt': '''
PROJCS["Monte Mario / Italy zone 1",
    GEOGCS["Monte Mario",
        DATUM["Monte_Mario",
            SPHEROID["International 1924",6378388,297,
                AUTHORITY["EPSG","7022"]],
            AUTHORITY["EPSG","6265"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.01745329251994328,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4265"]],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",9],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",1500000],
    PARAMETER["false_northing",0],
    AUTHORITY["EPSG","3003"],
    AXIS["X",EAST],
    AXIS["Y",NORTH]]
'''})


In [ ]:
rf_rate = r.createVariable('rainfall_rate', 'f4', 
                           (timed.name, xd.name, yd.name), 
                           fill_value=1.0e+20)

In [ ]:
setattr(rf_rate, {'long_name': 'estimated rainfall rate',
                  'standard_name': 'rainfall_rate',
                  'coordinates': 'lat lon',
                  'grid_mapping': 'crs',
                  'units': 'mm/hour'})

In [ ]:
x[:] = xpos
y[:] = ypos

In [ ]:
fill_value = rf_rate._FillValue
deltas = []
for ts, fname in radar_images[:10]:
    deltas.append(ts - first_tstamp)
    signal, mask = get_image_data(fname)
    rainfall = utils.estimate_rainfall(signal, mask)
    rainfall[mask == 0] = fill_value
    rf_rate[len(times), :, :] = rainfall

In [ ]:
times[:] = [_.seconds/60 for _ in deltas]

In [ ]:
r.close()